# Multichannel Source Separation

*This notebook uses [[Adel, 2012]](README.md) as reference.*

Multichannel source separation (MSS) involves separating the signals from from $N$ unobserved sources, given $M$ microphones. For a given environment, there can be many paths from each source to each microphone (see illustration below). The signal received by the $j^{th}$ microphone can be modeled by the following equation:

$$x_j(n) = \sum_{i=1}^{N} \sum_{p=1}^{P} h_{ji}^p(p) S_i(n - \Delta_{ji}^p) \quad \text{with} \quad j = 1 \ldots M$$
$$\begin{aligned}
\text{where: } \\
S_i \quad & i^{th} \text{ source signal} \\
x_j \quad & \text{signal received by the } j^{th} \text{ microphone that contains} \\
          & \text{a mixture of sounds from all the sources} \\
h_{ji}^p \quad & P \text{-point room impulse response from source } i \text{ to} \\
               & \text{ microphone } j \text{ along path } p \\
P \quad & \text{number of paths between each source-microphone pair} \\
\Delta_{ji}^p \quad & \text{time delay from source } i \text{ to microphone } j \text{ along path } p \\
\end{aligned}$$

![](images/multichannel_room_acoustics_model.png 'Multichannel Room Acoustics Model')
<br>*Multichannel Room Acoustics Model: [[Adel, 2012]](README.md)*

This model is an *echoic* model since it accounts for the reverberations that occur as the source signal propagates throughout the room. A reverberation free *anechoic* model can be obtained by setting the number of paths $P$ to 1. Doing so results in the following equation:

$$x_j(n) = \sum_{i=1}^{N} h_{ji} \, S_i(n - \Delta_{ji}) \quad \text{with} \quad j = 1 \ldots M$$

An even simpler model can be derived by assuming that all the source signals arrive at the microphones at the same time $\left(\Delta_{ji} = 0\right)$. This is called the *instantaneous* mixing model and is defined by the following equation:

$$x_j(n) = \sum_{i=1}^{N} h_{ji} \, S_i(n) \quad \text{with} \quad j = 1 \ldots M$$

## Beamforming

Beamforming is a spatio-temporal filtering technique that operates on the outputs of a microphone array. It can be thought of as an angular and frequency dependent filter that can be used to amplify the desired source while attenuating the undesired sources. This is accomplished by filtering the microphone signals and combining each of their outputs such that there is constructive interference of the desired source signal and destructive interference of the other signals.

Beamforming can be performed in either the time domain or the frequency domain. Beamforming in the time domain can be accomplished by applying a FIR filter to each of the microphone inputs and summing together the resulting signal. The estimated source signal $\hat{s}(t)$ can be obtained by computing the multichannel FIR filter $w_{i,p}$ in the following equation:

$$\hat{s}(t) = \sum_{i=1}^{N} \sum_{p=0}^{P-1} w_{i,p} \, x_i(t - p)$$

where $P-1$ is the number of delays in each of the $N$ filters. In frequency domain beamforming, the microphone signals are separated into narrowband frequency bins using the short time Fourier transform (STFT) and the data in each frequency bin is processed separately. 

Beamforming techniques can be classified as being either *data-dependent* or *data-independent*. Data-independent or *deterministic* beamformers have filters that do not depend on the signals received at the microphone inputs. Conversely, data-dependent beamformers use statistical optimization techniques to update their filters, improving the performance of the beamformer.

### Deterministic Beamforer

Since the filters in a deterministic beamformer do not depend on the microphone inputs, they must be selected to approximate a desired response. An example usecase for a deterministic beamformer could be that you know the position of the desired source ahead of time and you want to selectively isolate signals in the direction of the desired signal. Another usecase could be that you want to filter out certain frequencies from an undesired signal that is in a particular direction.

The most basic deterministic beamforming technique is called *delay-and-sum* beamforming. With delay-and-sum beamforming, each microphone signal is delayed so that there is constructive interference of the desired signal when all of the channels are summed together. The difference in signal arrival time at each microphone input can be approximated by taking the STFT of each microphone signal and comparing the respective phases in each frequency bin. An illustration of delay-and-sum beamforming is provided below.

*Delay-and-Sum Beamforming*<br>
![](images/delay_and_sum_beamforming.png 'Delay-and-Sum Beamforming')

In reverberant environments, the performance of the delay-and-sum beamformer can often be insufficient. To fix this issue, the delay can be replaced with a FIR filter as illustrated below. 

*Filter-and-Sum Beamforming*<br>
![](images/filter_and_sum_beamforming.png 'Filter-and-Sum Beamforming')


### Statistically Optimum Beamformer

A statistically optimum beamformer uses statistical properties of the desired and interfering signals to make the output of the beamformer optimum in some sense. This is done by trying to minimize the error between the beamformer output and the desired signal. There are several metrics that can be used when computing this error, some of which include the *maximum signal-to-noise ratio (MSNR)*, *minimum mean-squared error (MMSE)*, *minimum variance distortionless response (MVDR)* and *linear constraint minimum variance (LCMV)*.

For example, if we are trying to minimize the mean-squared error between the desired signal $d(n)$ and the output signal $y(n)$, we can derive the following cost function:

$$J = E \left[ \left| d(n) - y(n) \right|^2 \right]$$

where $E$ is the [*expectation operator*](https://en.wikipedia.org/wiki/Expected_value). Minimizing this cost function entails taking its gradient and setting it to 0. If we substitute the output of the beamformer $y(n) = w^H \, x(n)$ into this equation we get:

$$\nabla J \, = \, -2 r_{xd} \, + \, 2 R_{xx} \, w \, = \, 0$$

where $R_{xx} = E \left[ x(n) \, x^H(n) \right]$ is the correlation matrix of the input signal $x(n)$ and $r_{xd} = \left[ x(n) \, d^* (n)\right]$ is the cross-correlation vector between the microphone input and the desired signal. Solving this equation for $w$ gives us a formula for the optimum filter weights:

$$w_{MMSE} = R_{xx}^{-1} \, r_{xd}$$

This equation is known as the Wiener-Hopf equation or the [optimum Wiener solution](https://en.wikipedia.org/wiki/Wiener_filter).